# Seurat: data integration

In [1]:
library(Seurat)
library(SeuratData)
library(patchwork)
library(glmGamPoi)
library(ggplot2) #for plot saving

Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used

### Variable inputs

In [2]:
tissue <- "AO" # Options: AO,MO,WA

In [3]:
conditionnames <- list("C","ED","PL","IC","LD") # name of the conditions in the preferred order of integration and display. 
refdatasets <- c(1, 5) # datasets 
#Excluding the reference datasets, datasets will be integrated from left to right

In [ ]:
filenames <- paste0("integration-input/",tissue,"/",tissue,"_",conditionnames[],".rds")

### Loading data

In [ ]:
#load datasets as Seurat objects
mergedata.list <- lapply(filenames,readRDS)

In [ ]:
mergedata.list

### Preparation

In [ ]:
# normalize and identify variable features for each dataset independently
# do this with SCTransform, right????
mergedata.list <- lapply(X = mergedata.list, FUN = SCTransform, method = "glmGamPoi") #CHANGED for rpca: method = "glmGamPoi"

In [ ]:
features <- SelectIntegrationFeatures(object.list = mergedata.list, nfeatures = 3000)

In [ ]:
mergedata.list <- PrepSCTIntegration(object.list = mergedata.list, anchor.features = features)

## Integration

## CCA2

In [ ]:
# Anchors (cca 2)
assay.anchors.cca2 <- FindIntegrationAnchors(object.list = mergedata.list, reference = refdatasets, normalization.method = "SCT",anchor.features = features, reduction="cca",k.anchor=20,dims = 1:50)
# this command creates an 'integrated' data assay
assay.combined.sct.cca2 <- IntegrateData(anchorset = assay.anchors.cca2, normalization.method = "SCT")

In [ ]:
assay.combined.sct.cca2 <- RunPCA(assay.combined.sct.cca2, verbose = FALSE)
assay.combined.sct.cca2 <- RunUMAP(assay.combined.sct.cca2, reduction = "pca", dims = 1:30)

In [ ]:
saveRDS(assay.combined.sct.cca2, file = paste0("integration-input/",tissue,"/",tissue,"_all.rds"))

In [ ]:
# Visualization
p1 <- DimPlot(assay.combined.sct.cca2, reduction = "umap", group.by = "ConditionName")
# p1[[1]]$layers[[1]]$aes_params$alpha = .1 
p1
ggsave(paste0(tissue,"_umap_all_cca_2.png"), plot= p1)

In [ ]:
sidebyside = DimPlot(assay.combined.sct.cca2, reduction = "umap", split.by = "ConditionName", group.by = "ConditionName")
sidebyside
ggsave(paste0(tissue,"_umap_cca_2_side.png"),sidebyside)